# <center>Data Science Training</center>
<center><b>Model Creation Template</b><br>
2016</center>

In [1]:
import numpy as np
import pandas as pd

import sklearn as sk
from sklearn.metrics import log_loss
from sklearn.ensemble import BaggingClassifier 
from sklearn.externals import joblib
import xgboost as xgb

%matplotlib inline
import matplotlib.pyplot as plt
from __future__ import division
from __future__ import print_function

# Read file

In [2]:
dataset_modelA_clean = pd.read_csv("input_path",sep="|",na_values=["\N", "NULL"]) #input file path

# Get feature columns

In [4]:
ALL_TARGETS = ['target_all']

targeting_features_file = 'feature_path' #'feature_columns_metadata_path' 
feature_df_targeting = pd.read_csv(targeting_features_file,na_values=["\N","NULL"])

feature_columns = feature_df_targeting.targeting_features.tolist()

print(len(feature_columns))

In [6]:
# get data set for predicting
dataset_modelA_clean_targeting = dataset_modelA_clean[np.concatenate([feature_columns, ALL_TARGETS])].copy()

In [7]:
X_train = dataset_modelA_clean_targeting.drop(ALL_TARGETS, axis=1)

In [8]:
#check target percentage in population
print(ALL_TARGETS)

for target in ALL_TARGETS:
    print("target proportion %s"%target)
    print(dataset_modelA_clean_targeting[target].mean())
    print(dataset_modelA_clean_targeting[target].mean())

['target_all']
target proportion target_all
0.0215432748012
0.0215432748012


# Create targeting model and store it as pickle object

In [12]:
print('Start TARGETING MODELA CREATION WITH BAGGING OF XGB')

target = 'target_all'

clf = xgb.XGBClassifier(objective='binary:logistic', max_depth=6, n_estimators=50, learning_rate=0.05,
                        max_delta_step=1, min_child_weight=25, gamma=0.1, scale_pos_weight=0.85, colsample_bytree=0.85,
                        subsample=0.85, colsample_bylevel=1, nthread=10, seed=27)

#clfbag_targeting = BaggingClassifier(clf, n_estimators=10, max_samples=0.9, max_features=0.9)

train_X = X_train[feature_columns].drop(['svocmasterid'], axis=1).reset_index(drop=True)

y_train = np.array(dataset_modelA_clean_targeting[target].astype(np.uint8))

clf.fit(train_X, y_train)

print('STROING MODELA TARGETING MODEL TO PICKLE')
joblib.dump(clf,'model_savingpath', compress=1)
print('DONE!')


Start TARGETING MODELA CREATION WITH BAGGING OF XGB
STROING MODELA TARGETING MODEL TO PICKLE
DONE!


# Get Feature Importance

In [13]:
def get_importance(_bst, _importance_type):
    # if it's weight, then omap stores the number of missing values
    fmap = ''
    if _importance_type == 'weight':
        # do a simpler tree dump to save time
        trees = _bst.get_dump(fmap, with_stats=False)

        fmap = {}
        for tree in trees:
            for line in tree.split('\n'):
                # look for the opening square bracket
                arr = line.split('[')
                # if no opening bracket (leaf node), ignore this line
                if len(arr) == 1:
                    continue

                # extract feature name from string between []
                fid = arr[1].split(']')[0].split('<')[0]

                if fid not in fmap:
                    # if the feature hasn't been seen yet
                    fmap[fid] = 1
                else:
                    fmap[fid] += 1

        return fmap

    else:
        trees = _bst.get_dump(fmap, with_stats=True)

        _importance_type += '='
        fmap = {}
        gmap = {}
        for tree in trees:
            for line in tree.split('\n'):
                # look for the opening square bracket
                arr = line.split('[')
                # if no opening bracket (leaf node), ignore this line
                if len(arr) == 1:
                    continue

                # look for the closing bracket, extract only info within that bracket
                fid = arr[1].split(']')

                # extract gain or cover from string after closing bracket
                g = float(fid[1].split(_importance_type)[1].split(',')[0])

                # extract feature name from string before closing bracket
                fid = fid[0].split('<')[0]

                if fid not in fmap:
                    # if the feature hasn't been seen yet
                    fmap[fid] = 1
                    gmap[fid] = g
                else:
                    fmap[fid] += 1
                    gmap[fid] += g

        # calculate average value (gain/cover) for each feature
        for fid in gmap:
            gmap[fid] = gmap[fid] / fmap[fid]

        return gmap

In [14]:
feature_importances_data = []
features = feature_columns
for feature_name, feature_importance in get_importance(clf.booster(), 'gain').iteritems():
    feature_importances_data.append({
       'feature': feature_name,
       'importance': feature_importance
    })
    
feature_importances = pd.DataFrame(feature_importances_data)
feature_importances_sort = feature_importances.sort_values(by='importance',ascending=False)
feature_importances_top50 = feature_importances_sort[:50][::-1].reset_index(drop=True)

In [15]:
feature_importances['abs_imp'] = feature_importances['importance'].apply(lambda x: abs(x))
feature_importances_sort = feature_importances.sort_values(by='abs_imp',ascending=False)
feature_importances_sort['relative_imp'] = 100.0 * (feature_importances_sort['abs_imp'] / feature_importances_sort['abs_imp'].max())


feature_importances_sort = feature_importances_sort[::-1].reset_index(drop=True)
plt.figure(figsize=(10, 20))
plt.title("Feature importances for Model")
plt.barh(feature_importances_sort.index, feature_importances_sort['relative_imp'],
         color='#348ABD', align="center", lw='3', edgecolor='#348ABD', alpha=0.6)
plt.yticks(feature_importances_sort.index, feature_importances_sort['feature'], fontsize=12,)
plt.ylim([-1, feature_importances_sort.index.max()+1])
plt.xlim([0, feature_importances_sort['relative_imp'].max()*1.1])
plt.show()